# Select columns from Nanoimager csv and write a csv for ThunderSTORM

**ONI Nanoimager is a commercial widefield microscope which is used for localisation microscopy. The Nanoimager software has lots of cool features but you may still want to export the data for other analysis software. For example, I use ThunderSTORM (a very powerful localisation microscopy data analysis plugin for imageJ) to correct for drift using fiducials and to merge localisations of fluorophores which are on for multiple frames. To do so, I need the data exported from the Nanoimager software as a csv file in a format that can be read by ThunderSTORM. Unfortunatley, the current version of Nanoimager csv file cannot be directly read by ThunderSTORM. Here, in this notebook, I show how I create a ThunderSTORM friendly csv from the Nanoimager csv. In this notebook you will see how pandas can be used to easily manipulate tabular data and save it to a csv file.**

## Import data

In [1]:
# Here I import the python packages.
import pandas as pd
import numpy as np

In [2]:
# Load the localisation file from the nanoimager as a dataframe
df = pd.read_csv('Nanoimager_localizations.csv')

In [3]:
# Here I look at the top two rows of the dataframe.
df.head(2)

,Channel,Frame,X (nm),Y (nm),Z (nm),X precision (nm),Y precision (nm),Photons,Background,PSF Sigma X (pix),PSF Sigma Y (pix)
0,1,18,17393.363281,5341.088867,0.0,12.122814,14.760088,576.440552,45.888653,1.065249,1.301303
1,1,18,28469.585938,6503.035645,0.0,19.043039,8.590194,2415.484863,45.925720,1.979784,1.528099


In [4]:
# Here,I look at the column names, their datatypes and the number of rows.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485058 entries, 0 to 485057
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Channel            485058 non-null  int64  
 1   Frame              485058 non-null  int64  
 2   X (nm)             485058 non-null  float64
 3   Y (nm)             485058 non-null  float64
 4   Z (nm)             485058 non-null  float64
 5   X precision (nm)   485058 non-null  float64
 6   Y precision (nm)   485058 non-null  float64
 7   Photons            485058 non-null  float64
 8   Background         485058 non-null  float64
 9   PSF Sigma X (pix)  485058 non-null  float64
 10  PSF Sigma Y (pix)  485058 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 40.7 MB


## Format data

### Choose the colour channel that you want to analyse

In [5]:
# Here I only want channel 1 or the red channel
df=df[df.Channel == 1] # filter channel 1, use 0 for the green channel
df.shape # In this case we only had channel 1 so no change in number of rows.

(485058, 11)

### Create a column that stores the localisation id

In [6]:
# Here I create a column named id and insert it to the start of the dataframe
id = np.arange(start=1, stop=df.shape[0]+1, step=1)
id = pd.Series(id) # convert numpy array to pandas series

In [7]:
# insert the column at the start of the dataframe
df.insert(0, "id", id, True) 

In [8]:
# check the increase in the number of columns in the dataframe to show successful insertion
df.shape

(485058, 12)

In [9]:
# check the position of the insertion
df.head(2) # id is the first column

,id,Channel,Frame,X (nm),Y (nm),Z (nm),X precision (nm),Y precision (nm),Photons,Background,PSF Sigma X (pix),PSF Sigma Y (pix)
0,1,1,18,17393.363281,5341.088867,0.0,12.122814,14.760088,576.440552,45.888653,1.065249,1.301303
1,2,1,18,28469.585938,6503.035645,0.0,19.043039,8.590194,2415.484863,45.925720,1.979784,1.528099


### For using the ThunderSTORM imageJ plugin for further analysis, I do not need all  the columns in the Nanoimager csv. Here, I select the required columns: id, Frame, X(nm), Y(nm), Photons and Background

#### Method 1

In [10]:
# You can use the column numbers to make the selection
columns_selected=[1,3,4,5,9,10]
columns_selected=np.array(columns_selected)-1
df_selected=df[df.columns[columns_selected]]


df_selected.head(2)#check if the correct columns were selected

,id,Frame,X (nm),Y (nm),Photons,Background
0,1,18,17393.363281,5341.088867,576.440552,45.888653
1,2,18,28469.585938,6503.035645,2415.484863,45.925720


#### Method 2

In [11]:
# Or you can use the column names directly
df_selected=df[['id', 'Frame', 'X (nm)', 'Y (nm)', 'Photons', 'Background']]
#df_selected=df[["id", "Frame", "X (nm)", "Y (nm)", "Photons", "Background"]]
df_selected.head(2)#check if the correct columns were selected

,id,Frame,X (nm),Y (nm),Photons,Background
0,1,18,17393.363281,5341.088867,576.440552,45.888653
1,2,18,28469.585938,6503.035645,2415.484863,45.925720


### Here I change the column names to a ThunderSTORM friendly format.

In [12]:
df_selected.columns = ['id', 'frame','x [nm]','y [nm]','intensity [photon]','bkgstd [photon]']
df_selected.head(2) # Check that column names have been changed

,id,frame,x [nm],y [nm],intensity [photon],bkgstd [photon]
0,1,18,17393.363281,5341.088867,576.440552,45.888653
1,2,18,28469.585938,6503.035645,2415.484863,45.925720


## Export data to ThunderSTORM csv

In [13]:
# Write the localisation file csv for ThunderSTORM
df_selected.to_csv('ThunderSTORM_localizations.csv')